In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import uuid
import unicodedata


# Leer el archivo Excel
df = pd.read_excel('data_carga.xlsx')



# Limpieza básica de columnas
df.columns = df.columns.str.strip().str.lower()
df['tipo_repuesto'] = df['tipo_repuesto'].str.strip().str.lower()
df['pagina'] = df['pagina'].str.lower()
# df['pagina'] = df['pagina'].replace('mundorepuestos', 'Mundo Repuestos')
df['modelo'] = df['modelo'].str.strip()

# Conexión a PostgreSQL
usuario = 'postgres'
contraseña = 'jmms'
host = 'localhost'
puerto = '5432'
nombre_bd = 'Repuestos'

engine = create_engine(f"postgresql://{usuario}:{contraseña}@{host}:{puerto}/{nombre_bd}")

# Verificar conexión
with engine.connect() as conn:
    result = conn.execute(text("SELECT version();"))
    print("Conectado a:", result.fetchone())

# Cargar datos de referencia
df_tiendas = pd.read_sql("SELECT id_tienda, nombre FROM tienda", engine)
df_modelos = pd.read_sql("SELECT id_modelo, nombre FROM modelo", engine)
df_tipos = pd.read_sql("SELECT id_tipo, nombre FROM tipo_repuesto", engine)

def quitar_tildes(texto):
    if isinstance(texto, str):
        return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    return texto

# Aplicar a la columna 'nombre' de tipo_repuesto
df_tipos['nombre'] = df_tipos['nombre'].apply(quitar_tildes).str.strip().str.lower()

# Normalizar nombres
df_tiendas['nombre'] = df_tiendas['nombre'].str.strip().str.lower()
df_modelos['nombre'] = df_modelos['nombre'].str.strip()
df_tipos['nombre'] = df_tipos['nombre'].str.strip().str.lower()

# Buscar id_tienda con contains sobre 'pagina'
def encontrar_id_tienda(pagina_val):
    for _, fila in df_tiendas.iterrows():
        if pd.notna(pagina_val) and fila['nombre'] in pagina_val:
            return fila['id_tienda']
    return None

df['id_tienda'] = df['pagina'].apply(encontrar_id_tienda)

# Merge exacto por modelo
df_modelos = df_modelos.rename(columns={"nombre": "modelo"})
df = df.merge(df_modelos, on="modelo", how="left")

# Match parcial para tipo_repuesto
def encontrar_tipo_id(valor):
    for _, fila in df_tipos.iterrows():
        if pd.notna(valor) and fila['nombre'] in valor:
            return fila['id_tipo']
    return None

df['id_tipo'] = df['tipo_repuesto'].apply(encontrar_tipo_id)

# Generar columna id_repuesto única de máximo 20 caracteres
def generar_id_repuesto():
    return str(uuid.uuid4()).replace("-", "")[:20]

df['id_repuesto'] = [generar_id_repuesto() for _ in range(len(df))]
while df['id_repuesto'].duplicated().any():
    duplicados = df['id_repuesto'].duplicated(keep=False)
    df.loc[duplicados, 'id_repuesto'] = [generar_id_repuesto() for _ in range(duplicados.sum())]


# Construir el DataFrame final
df = df[[
    'id_repuesto', 'nombre', 'id_tienda', 'id_modelo', 'id_tipo',
    'precio', 'link', 'imagen', 'descripcion', 'fecha_carga'
]]

df = df.dropna(subset=['id_tipo'])
df = df.dropna(subset=['id_tienda']) #QUITAR ESTO

df.to_sql('repuestos', engine, if_exists='append', index=False)



Conectado a: ('PostgreSQL 15.1, compiled by Visual C++ build 1914, 64-bit',)


103